<a href="https://colab.research.google.com/github/gupta-keshav/age_gender_classification/blob/master/AdienceGender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/datasetTF.rar", outdir="/content")

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model
input_shape = (224, 224, 3)
vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
                                     input_shape=input_shape)   

In [0]:
last = vgg.output

In [0]:
x = Flatten()(last)
x = Dropout(0.3)(x)
x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)
x = Dropout(0.55)(x)

In [0]:
preds = Dense(1, activation = 'sigmoid')(x)

In [0]:
model = Model(vgg.input, preds)

In [0]:
import pandas as pd
layers = [(layer.output, layer.name, layer.trainable) for layer in model.layers]
pd.set_option('max_colwidth', -1)

pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

In [0]:
for i in range(17):
  model.layers[i].trainable = False

In [0]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
        '/content/datasetTF/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
validate_generator = train_datagen.flow_from_directory(
        '/content/datasetTF/validate',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')


In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=440, epochs=1, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=0, use_multiprocessing=True, shuffle=True)